In [2]:
import pandas as pd
import numpy as np
import env

**Get data**

In [8]:
# get access url
api_url = env.get_db_access("logs")
cohorts_url = env.get_db_access("curriculum_logs")

# read api_access data
codeup_api = pd.read_sql('SELECT * FROM api_access', api_url)
codeup_cohorts = pd.read_sql('SELECT * FROM cohorts', cohorts_url)
codeup_logs = pd.read_sql('SELECT * FROM logs', cohorts_url)

In [13]:
codeup_api.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [12]:
codeup_cohorts.head()

,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1


In [14]:
codeup_logs.head()

,date,time,path,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [27]:
codeup_cohorts.shape

(53, 9)

In [28]:
codeup_logs.shape

(900223, 6)

**clean data**

In [22]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [23]:
codeup_api = codeup_api.entry.apply(parse_log_entry) # Applying the function to the entire dataframe
codeup_api.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


**Merge data**

In [30]:
# merge the cohorts data with the logs data
codeup = codeup_cohorts.merge(codeup_logs, left_on="id", right_on="cohort_id", how="right")
codeup.head()

,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,date,time,path,user_id,cohort_id,ip
0,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


**Check nulls**

In [31]:
codeup.cohort_id.isna().sum()

52893

In [32]:
codeup.user_id.isna().sum()

0

In [34]:
codeup.date.unique().shape

(1182,)

**Check data types**

In [35]:
codeup.dtypes

id            float64
name           object
slack          object
start_date     object
end_date       object
created_at     object
updated_at     object
deleted_at     object
program_id    float64
date           object
time           object
path           object
user_id         int64
cohort_id     float64
ip             object
dtype: object

**wrangle function**

In [36]:
def wrangle_codeup():
    """
    Goal: To return clean cohort and logs data from codeup data base for exploraton
    """
    # get access url
    # api_url = env.get_db_access("logs")
    cohorts_url = env.get_db_access("curriculum_logs")

    # read api_access data
    # codeup_api = pd.read_sql('SELECT * FROM api_access', api_url)
    codeup_cohorts = pd.read_sql('SELECT * FROM cohorts', cohorts_url)
    codeup_logs = pd.read_sql('SELECT * FROM logs', cohorts_url)
    
    # merge the cohorts data with the logs data
    codeup = codeup_cohorts.merge(codeup_logs, left_on="id", right_on="cohort_id", how="right")
    return codeup